In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movielens-20m-dataset/rating.csv
/kaggle/input/movielens-20m-dataset/link.csv
/kaggle/input/movielens-20m-dataset/genome_tags.csv
/kaggle/input/movielens-20m-dataset/genome_scores.csv
/kaggle/input/movielens-20m-dataset/tag.csv
/kaggle/input/movielens-20m-dataset/movie.csv


In [2]:
#Load Dataset
movies_df=pd.read_csv('/kaggle/input/movielens-20m-dataset/movie.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
#Remove the year from title and add onto a separate column
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies_df['year']=movies_df.year.str.extract('(\d\d\d\d)',expand=False)
movies_df['title']=movies_df.title.str.replace('(\(\d\d\d\d\))','')
movies_df['title']=movies_df.title.apply(lambda x:x.strip())
#movies_df['title']=movies_df.title.str.strip() is an alternative 
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [4]:
#split the title string based on '|' character
movies_df['genres']=movies_df.genres.apply(lambda x:x.split('|'))
#movies_df['genres']=movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [5]:
#copy the movie datarframe
movies_with_genres_df=movies_df.copy()
movies_with_genres_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [6]:
#iterate through the dataframe 
for index, row in movies_with_genres_df.iterrows():
    for genre in row['genres']:
        movies_with_genres_df.at[index, genre] = 1
#fill the na with 0
movies_with_genres_df=movies_with_genres_df.fillna(0)
movies_with_genres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
#Give the user ratings 
user_input=[{'title':'Interstellar','rating':3.5},
           {'title':'Spider-Man','rating':5},
           {'title':'Toy Story','rating':3},
           {'title':'Inception','rating':2},
           {'title':'Man of Steel','rating':2}]
input_movies=pd.DataFrame(user_input)
input_movies.head()

,title,rating
0,Interstellar,3.5
1,Spider-Man,5.0
2,Toy Story,3.0
3,Inception,2.0
4,Man of Steel,2.0


In [8]:
#Identify the user input in the movie dataframe based on the tite 
user_movies=movies_with_genres_df[movies_with_genres_df['title'].isin(input_movies['title'].tolist())] 
user_movies=user_movies.fillna(0)
user_movies.head(10)

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5252,5349,Spider-Man,"[Action, Adventure, Sci-Fi, Thriller]",2002,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15534,79132,Inception,"[Action, Crime, Drama, Mystery, Sci-Fi, Thrill...",2010,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
21134,103042,Man of Steel,"[Action, Adventure, Fantasy, Sci-Fi, IMAX]",2013,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
22919,109487,Interstellar,"[Sci-Fi, IMAX]",2014,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
#Remove Unwanted columns
user_movies=user_movies.drop('title',1).drop('movieId',1).drop('year',1).drop('genres',1)
user_movies=user_movies.reset_index(drop=True)
user_movies

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
user_movies.shape

(5, 20)

In [11]:
input_movies['rating']

0    3.5
1    5.0
2    3.0
3    2.0
4    2.0
Name: rating, dtype: float64

In [12]:
#Create a user profile based on the ratings.It contains genre wise weights,high weights correspond to higher affinity
user_profile=user_movies.transpose().dot(input_movies['rating'])
user_profile

Adventure             10.5
Animation              3.5
Children               3.5
Comedy                 3.5
Fantasy                5.5
Romance                0.0
Drama                  3.0
Action                10.0
Crime                  3.0
Thriller               8.0
Horror                 0.0
Mystery                3.0
Sci-Fi                12.0
IMAX                   7.0
Documentary            0.0
War                    0.0
Musical                0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

In [13]:
#remove the unwanted columns from the movie dataframe as well
genre_table=movies_with_genres_df.set_index(movies_with_genres_df['movieId'])
genre_table=genre_table.drop('movieId',1).drop('title',1).drop('genres',1).drop('year',1)
genre_table.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
genre_table.shape

(27278, 20)

In [15]:
#Each movie will be given a certain value based on the user atings 
recommendation_table_df=((genre_table*user_profile).sum(axis=1))/(user_profile.sum())
recommendation_table_df.head(20)

movieId
1     0.365517
2     0.268966
3     0.048276
4     0.089655
5     0.048276
6     0.289655
7     0.048276
8     0.193103
9     0.137931
10    0.393103
11    0.089655
12    0.048276
13    0.241379
14    0.041379
15    0.282759
16    0.082759
17    0.041379
18    0.048276
19    0.048276
20    0.379310
dtype: float64

In [16]:
#Sort these values 
recommendation_table_df=recommendation_table_df.sort_values(ascending=False)
recommendation_table_df.head()

movieId
85261    0.689655
71999    0.675862
5018     0.668966
52462    0.662069
77561    0.655172
dtype: float64

In [17]:
#Recommend the first 20 movies based on these values
recommended_movies=movies_df[movies_df['movieId'].isin(recommendation_table_df.head(20).keys())]
recommended_movies

,movieId,title,genres,year
542,546,Super Mario Bros.,"[Action, Adventure, Children, Comedy, Fantasy,...",1993
4685,4781,Megiddo: The Omega Code 2,"[Action, Adventure, Fantasy, Sci-Fi, Thriller]",2001
4922,5018,Motorama,"[Adventure, Comedy, Crime, Drama, Fantasy, Mys...",1991
6251,6350,Laputa: Castle in the Sky (Tenkû no shiro Rapy...,"[Action, Adventure, Animation, Children, Fanta...",1986
6260,6365,"Matrix Reloaded, The","[Action, Adventure, Sci-Fi, Thriller, IMAX]",2003
6792,6902,Interstate 60,"[Adventure, Comedy, Drama, Fantasy, Mystery, S...",2002
6822,6934,"Matrix Revolutions, The","[Action, Adventure, Sci-Fi, Thriller, IMAX]",2003
8939,26590,G.I. Joe: The Movie,"[Action, Adventure, Animation, Children, Fanta...",1987
9213,27155,"Batman/Superman Movie, The","[Action, Adventure, Animation, Children, Fanta...",1998
9819,32031,Robots,"[Adventure, Animation, Children, Comedy, Fanta...",2005
